In [24]:
!pip install duckduckgo_search
from duckduckgo_search import DDGS # Import DDGS

def fetch_news_duckduckgo(query, max_results=5):
    headlines = []
    with DDGS() as ddgs:
        for r in ddgs.news(query, max_results=max_results):
            headlines.append(r['title'])
    return headlines # This line was incorrectly indented

# Remove the duplicated code block:
# for r in ddgs.news(query, max_results=max_results):
#     headlines.append(r['title'])
# return headlines

fetch_news_duckduckgo('apple shares')

['Apple shares slump as tariffs take toll on iPhone maker',
 'Apple shares fall as tariff costs to add more agony',
 "Apple's first bond offering in two years headlines busy primary",
 'Apple (AAPL) adds $100 billion to its share buyback program and ups dividend payout again',
 'Warren Buffett hails Tim Cook for making Berkshire more money than he has — after selling two-thirds of his Apple stake']

In [25]:
!pip install duckduckgo_search
from duckduckgo_search import DDGS

In [26]:
pip install feedparser

In [28]:
import feedparser

def get_google_news_rss(company):
    feed_url = f"https://news.google.com/rss/search?q={company}+stock"
    feed = feedparser.parse(feed_url)
    return [entry['title'] for entry in feed.entries[:10]]

In [29]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

Device set to use cpu


In [31]:
results = []
for ticker, headlines in d1.items():
    for headline in headlines:
        result = pipe(headline)  # Pass individual headline to the pipeline
        # Check if result is a list and convert to dictionary
        if isinstance(result, list):
            result = result[0]  # Assuming the first element is the relevant dictionary
        results.append({"ticker": ticker, "headline": headline, **result})  # Store ticker, headline, and result

# Aggregate sentiment
sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}
for result in results:
    label = result["label"].lower()
    sentiment_counts[label] += 1
    print(f"{result['ticker']} - {result['headline']}\n   ➤ Sentiment: {result['label']} (Confidence: {round(result['score'], 3)})\n")

# Show overall sentiment
total = sum(sentiment_counts.values())
print("=== Overall Apple Sentiment Summary ===")  # Assuming you want sentiment for AAPL
for sentiment, count in sentiment_counts.items():
    print(f"{sentiment.capitalize()}: {count} ({round(count / total * 100, 1)}%)")

overall = max(sentiment_counts, key=sentiment_counts.get)
print(f"\n📊 Overall Market Sentiment on {ticker}: **{overall.upper()}**")

NameError: name 'd1' is not defined

In [32]:
from transformers import pipeline
import yfinance as yf

# Load FinBERT sentiment analysis pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

def fetch_company_news(tickers):
    """
    Fetches the latest news headlines for a list of companies.

    Args:
    tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'TSLA', 'AMZN']).

    Returns:
    dict: A dictionary where the keys are ticker symbols and the values are lists of news headlines.
    """
    news_dict = {}

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        news_data = ticker.news

        if news_data:
            headlines = [article.get("title", "") for article in news_data]
            news_dict[ticker_symbol] = headlines
        else:
            news_dict[ticker_symbol] = ["No recent news found."]

    return news_dict

def analyze_sentiment_for_all_companies(tickers):
    """
    Analyzes sentiment for the news of multiple companies and returns an overall sentiment for each company.

    Args:
    tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'TSLA', 'AMZN']).

    Returns:
    dict: A dictionary where the keys are ticker symbols and the values are the overall sentiment ('positive', 'neutral', 'negative').
    """
    news_dict = fetch_company_news(tickers)

    sentiment_results = {}

    for ticker, headlines in news_dict.items():
        sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}

        # Analyze sentiment for each headline
        results = pipe(headlines)

        for result in results:
            label = result["label"].lower()
            sentiment_counts[label] += 1

        # Determine overall sentiment
        total = sum(sentiment_counts.values())
        overall_sentiment = max(sentiment_counts, key=sentiment_counts.get)

        sentiment_results[ticker] = {
            "positive": sentiment_counts["positive"],
            "neutral": sentiment_counts["neutral"],
            "negative": sentiment_counts["negative"],
            "overall_sentiment": overall_sentiment.upper()
        }

    return sentiment_results

# List of tickers to analyze
ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]

# Perform sentiment analysis for all companies in the ticker list
sentiment_analysis = analyze_sentiment_for_all_companies(ticker_list)

# Print the overall sentiment for each company
for ticker, sentiment_data in sentiment_analysis.items():
    print(f"--- Sentiment Analysis for {ticker} ---")
    print(f"Positive: {sentiment_data['positive']}")
    print(f"Neutral: {sentiment_data['neutral']}")
    print(f"Negative: {sentiment_data['negative']}")
    print(f"Overall Sentiment: {sentiment_data['overall_sentiment']}")
    print()


Device set to use cpu


--- Sentiment Analysis for AAPL ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for MSFT ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for GOOGL ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for AMZN ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for TSLA ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for META ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for NVDA ---
Positive: 0
Neutral: 1
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for BRK-B ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for JPM ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

--- Sentiment Analysis for V ---
Positive: 0
Neutral: 10
Negative: 0
Overall Sentiment: NEUTRAL

---

In [33]:
pip install gradio


In [34]:
from transformers import pipeline
import yfinance as yf
import gradio as gr

# Load FinBERT pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

# Global headlines store
all_headlines = {}

# Ticker list
ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]

def fetch_company_news(tickers):
    global all_headlines
    news_dict = {}

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        news_data = ticker.news

        if news_data:
            headlines = [article.get("title", "") for article in news_data]
        else:
            headlines = ["No recent news found."]

        news_dict[ticker_symbol] = headlines

    all_headlines = news_dict
    return news_dict

def analyze_single_ticker_sentiment(ticker_symbol):
    if ticker_symbol not in all_headlines:
        fetch_company_news([ticker_symbol])

    headlines = all_headlines.get(ticker_symbol, ["No recent news found."])

    if headlines == ["No recent news found."]:
        return "No recent news available.", ""

    sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}
    results = pipe(headlines)

    output_lines = []
    for i, (headline, result) in enumerate(zip(headlines, results), 1):
        label = result["label"].lower()
        sentiment_counts[label] += 1
        output_lines.append(f"{i}. {headline}\n   ➤ Sentiment: {result['label']} (Confidence: {round(result['score'], 3)})")

    total = sum(sentiment_counts.values())
    overall = max(sentiment_counts, key=sentiment_counts.get)

    summary = f"📊 Overall Sentiment for {ticker_symbol}: {overall.upper()}\n"
    summary += f"Positive: {sentiment_counts['positive']}  |  Neutral: {sentiment_counts['neutral']}  |  Negative: {sentiment_counts['negative']}"

    return "\n\n".join(output_lines), summary

# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("# 📰 Stock News Sentiment Analysis with FinBERT")

    with gr.Row():
        ticker_input = gr.Dropdown(choices=ticker_list, label="Select a Stock Ticker")
        analyze_button = gr.Button("Analyze")

    sentiment_output = gr.Textbox(label="News Headlines & Sentiments", lines=15)
    summary_output = gr.Textbox(label="Overall Sentiment Summary")

    analyze_button.click(
        fn=analyze_single_ticker_sentiment,
        inputs=[ticker_input],
        outputs=[sentiment_output, summary_output]
    )

# Launch the app
app.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1bf632a37e9ba42f7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [35]:
from transformers import pipeline
import yfinance as yf
import gradio as gr

# Load FinBERT pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

# Global headlines store
all_headlines = {}

# Ticker list
ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]

def fetch_company_news(tickers):
    global all_headlines
    news_dict = {}

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        news_data = ticker.news

        if news_data:
            headlines = [article.get("title", "") for article in news_data]
        else:
            headlines = ["No recent news found."]

        news_dict[ticker_symbol] = headlines

    all_headlines = news_dict
    return news_dict

def analyze_single_ticker_sentiment(ticker_symbol):
    if ticker_symbol not in all_headlines:
        fetch_company_news([ticker_symbol])

    headlines = all_headlines.get(ticker_symbol, ["No recent news found."])

    if headlines == ["No recent news found."]:
        return "No recent news available.", ""

    sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}
    results = pipe(headlines)

    output_lines = []
    for i, (headline, result) in enumerate(zip(headlines, results), 1):
        label = result["label"].lower()
        sentiment_counts[label] += 1
        output_lines.append(f"**{i}. {headline}**\n   ➤ Sentiment: {result['label']} (Confidence: {round(result['score'], 3)})")

    total = sum(sentiment_counts.values())
    overall = max(sentiment_counts, key=sentiment_counts.get)

    summary = f"📊 **Overall Sentiment for {ticker_symbol}: {overall.upper()}**\n"
    summary += f"Positive: {sentiment_counts['positive']}  |  Neutral: {sentiment_counts['neutral']}  |  Negative: {sentiment_counts['negative']}"

    return "\n\n".join(output_lines), summary

# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("# 📰 Stock News Sentiment Analysis with FinBERT")

    with gr.Row():
        ticker_input = gr.Dropdown(choices=ticker_list, label="Select a Stock Ticker")
        analyze_button = gr.Button("Analyze")

    sentiment_output = gr.Textbox(label="News Headlines & Sentiments", lines=15)
    summary_output = gr.Textbox(label="Overall Sentiment Summary")

    analyze_button.click(
        fn=analyze_single_ticker_sentiment,
        inputs=[ticker_input],
        outputs=[sentiment_output, summary_output]
    )

# Launch the app
app.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bfcb7dce70581f3b7a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [36]:
import yfinance as yf

# List of tickers
ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]

def fetch_profit_loss(tickers, period='1y'):
    """
    Fetches profit or loss percentage for a list of stock tickers over a given period.

    Args:
    tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'MSFT', 'AMZN']).
    period (str): Time period for calculating profit/loss ('1d', '5d', '1mo', '3mo', '1y').

    Returns:
    dict: A dictionary with tickers as keys and profit/loss percentages as values.
    """
    profit_loss_dict = {}

    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)

        # Get historical market data for the desired period
        hist_data = ticker.history(period=period)

        # Get the opening price from the start of the period and the current closing price
        start_price = hist_data['Close'].iloc[0]
        end_price = hist_data['Close'].iloc[-1]

        # Calculate the percentage change (Profit or Loss)
        profit_loss_percentage = ((end_price - start_price) / start_price) * 100
        profit_loss_dict[ticker_symbol] = profit_loss_percentage

    return profit_loss_dict

# Fetch profit/loss for all companies in the ticker list over 1 year
profit_loss_percentage = fetch_profit_loss(ticker_list, period='3mo')

# Print the profit/loss percentage for each company
for ticker, pl_percentage in profit_loss_percentage.items():
    print(f"{ticker}: {pl_percentage:.2f}%")


AAPL: -13.50%
MSFT: 6.17%
GOOGL: -13.55%
AMZN: -20.70%
TSLA: -25.21%
META: -14.70%
NVDA: -8.21%
BRK-B: 8.99%
JPM: -5.49%
V: 0.40%
PG: -5.47%
JNJ: 1.26%
UNH: -24.03%
HD: -11.54%
MA: -0.39%
DIS: -15.89%
PYPL: -13.01%
NFLX: 12.85%
INTC: 3.87%
PEP: -8.25%


In [37]:
import yfinance as yf
import pandas as pd

ticker_list = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "BRK-B", "JPM", "V",
    "PG", "JNJ", "UNH", "HD", "MA", "DIS", "PYPL", "NFLX", "INTC", "PEP"
]

ebitda_margins = {}

for ticker_symbol in ticker_list:
    ticker = yf.Ticker(ticker_symbol)

    try:
        financials = ticker.quarterly_financials

        # Extract revenue and operating income
        revenue = financials.loc['Total Revenue']
        operating_income = financials.loc['Operating Income']

        # Handle depreciation safely
        if 'Depreciation' in financials.index:
            depreciation_amortization = financials.loc['Depreciation']
        else:
            # If missing, assume 0 (safe fallback)
            depreciation_amortization = pd.Series(0, index=revenue.index)

        # Calculate EBITDA and Margin
        ebitda = operating_income + depreciation_amortization
        ebitda_margin = (ebitda / revenue) * 100

        # Store latest margin
        ebitda_margins[ticker_symbol] = ebitda_margin.iloc[0]

    except Exception as e:
        ebitda_margins[ticker_symbol] = f"Error: {e}"

# Convert to DataFrame
df = pd.DataFrame.from_dict(ebitda_margins, orient='index', columns=['EBITDA Margin (%)'])
print(df)


               EBITDA Margin (%)
AAPL                   31.029059
MSFT                   45.671224
GOOGL                  33.918479
AMZN                   11.823315
TSLA                     2.54978
META                   41.487451
NVDA                   61.107015
BRK-B  Error: 'Operating Income'
JPM    Error: 'Operating Income'
V                      67.073171
PG                     23.048139
JNJ                    28.771754
UNH                     7.710774
HD                     11.321277
MA                     59.324138
DIS                    16.516808
PYPL                   20.485175
NFLX                   31.746772
INTC                   -1.144707
PEP                    14.414867


In [38]:
import gradio as gr
from transformers import pipeline
import yfinance as yf
import pandas as pd

# Load FinBERT sentiment pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

# Mapping of full company names to tickers
company_dict = {
    "Apple Inc.": "AAPL",
    "Microsoft Corporation": "MSFT",
    "Alphabet Inc. (Google)": "GOOGL",
    "Amazon.com, Inc.": "AMZN",
    "Tesla, Inc.": "TSLA",
    "Meta Platforms, Inc.": "META",
    "NVIDIA Corporation": "NVDA",
    "Berkshire Hathaway Inc.": "BRK-B",
    "JPMorgan Chase & Co.": "JPM",
    "Visa Inc.": "V",
    "Procter & Gamble Co.": "PG",
    "Johnson & Johnson": "JNJ",
    "UnitedHealth Group Incorporated": "UNH",
    "The Home Depot, Inc.": "HD",
    "Mastercard Incorporated": "MA",
    "The Walt Disney Company": "DIS",
    "PayPal Holdings, Inc.": "PYPL",
    "Netflix, Inc.": "NFLX",
    "Intel Corporation": "INTC",
    "PepsiCo, Inc.": "PEP"
}

# Fetch news headlines for a ticker
def fetch_news(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    news_data = ticker.news
    return [article.get("title", "") for article in news_data] if news_data else []

# Analyze sentiment of headlines
def analyze_sentiment(headlines):
    sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}
    if headlines:
        results = pipe(headlines)
        for result in results:
            label = result["label"].lower()
            sentiment_counts[label] += 1
        overall_sentiment = max(sentiment_counts, key=sentiment_counts.get).upper()
    else:
        overall_sentiment = "NO DATA"
    return sentiment_counts, overall_sentiment

# Calculate profit/loss
def fetch_profit_loss(ticker, period='3mo'):
    hist_data = yf.Ticker(ticker).history(period=period)
    if hist_data.empty:
        return None
    start_price = hist_data['Close'].iloc[0]
    end_price = hist_data['Close'].iloc[-1]
    return ((end_price - start_price) / start_price) * 100

# Calculate EBITDA Margin (latest quarter)
def fetch_ebitda_margin(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        financials = ticker.quarterly_financials

        revenue = financials.loc['Total Revenue']
        operating_income = financials.loc['Operating Income']

        if 'Depreciation' in financials.index:
            depreciation_amortization = financials.loc['Depreciation']
        else:
            depreciation_amortization = pd.Series(0, index=revenue.index)

        ebitda = operating_income + depreciation_amortization
        ebitda_margin = (ebitda / revenue) * 100
        return ebitda_margin.iloc[0]

    except Exception as e:
        return None

# Combined Gradio function
def analyze_company(company_name):
    ticker = company_dict[company_name]
    headlines = fetch_news(ticker)
    sentiment_counts, overall_sentiment = analyze_sentiment(headlines)
    profit_loss = fetch_profit_loss(ticker)
    ebitda_margin = fetch_ebitda_margin(ticker)

    sentiment_output = (
        f"📰 **Sentiment Analysis for {company_name} ({ticker})**\n"
        f"Positive: {sentiment_counts['positive']}\n"
        f"Neutral: {sentiment_counts['neutral']}\n"
        f"Negative: {sentiment_counts['negative']}\n"
        f"Overall: {overall_sentiment}\n"
    )

    pl_output = (
        f"\n💹 **Profit/Loss for last 3 months**: "
        f"{profit_loss:.2f}%" if profit_loss is not None else "\n💹 No price data available."
    )

    ebitda_output = (
        f"\n📈 **Latest EBITDA Margin**: {ebitda_margin:.2f}%" if ebitda_margin is not None else "\n📈 EBITDA margin not available."
    )

    news_output = "\n\n🗞 **Top News Headlines:**\n" + "\n".join([f"- {h}" for h in headlines[:5]]) if headlines else "\n\n🗞 No recent headlines found."

    return sentiment_output + pl_output + ebitda_output + news_output

# Gradio app
with gr.Blocks() as demo:
    gr.Markdown("## 📊 Company Market Sentiment & Financial Performance Tracker")
    company_dropdown = gr.Dropdown(
        choices=list(company_dict.keys()),
        label="Select a Company",
        interactive=True
    )
    output_box = gr.Textbox(label="Analysis Output", lines=20, interactive=False)
    company_dropdown.change(fn=analyze_company, inputs=company_dropdown, outputs=output_box)

demo.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://03c0e253bb3943c691.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [39]:
import gradio as gr
from transformers import pipeline
import yfinance as yf
import pandas as pd

# Load FinBERT sentiment pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

# Mapping of full company names to tickers
company_dict = {
    "Apple Inc.": "AAPL",
    "Microsoft Corporation": "MSFT",
    "Alphabet Inc. (Google)": "GOOGL",
    "Amazon.com, Inc.": "AMZN",
    "Tesla, Inc.": "TSLA",
    "Meta Platforms, Inc.": "META",
    "NVIDIA Corporation": "NVDA",
    "Berkshire Hathaway Inc.": "BRK-B",
    "JPMorgan Chase & Co.": "JPM",
    "Visa Inc.": "V",
    "Procter & Gamble Co.": "PG",
    "Johnson & Johnson": "JNJ",
    "UnitedHealth Group Incorporated": "UNH",
    "The Home Depot, Inc.": "HD",
    "Mastercard Incorporated": "MA",
    "The Walt Disney Company": "DIS",
    "PayPal Holdings, Inc.": "PYPL",
    "Netflix, Inc.": "NFLX",
    "Intel Corporation": "INTC",
    "PepsiCo, Inc.": "PEP"
}

# Fetch news headlines for a ticker
def fetch_news(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    news_data = ticker.news
    return [article.get("title", "") for article in news_data] if news_data else []

# Analyze sentiment of headlines
def analyze_sentiment(headlines):
    sentiment_counts = {"positive": 0, "neutral": 0, "negative": 0}
    if headlines:
        results = pipe(headlines)
        for result in results:
            label = result["label"].lower()
            sentiment_counts[label] += 1
        overall_sentiment = max(sentiment_counts, key=sentiment_counts.get).upper()
    else:
        overall_sentiment = "NO DATA"
    return sentiment_counts, overall_sentiment

# Calculate profit/loss
def fetch_profit_loss(ticker, period='3mo'):
    hist_data = yf.Ticker(ticker).history(period=period)
    if hist_data.empty:
        return None
    start_price = hist_data['Close'].iloc[0]
    end_price = hist_data['Close'].iloc[-1]
    return ((end_price - start_price) / start_price) * 100

# Calculate EBITDA Margin
def fetch_ebitda_margin(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        financials = ticker.quarterly_financials

        revenue = financials.loc['Total Revenue']
        operating_income = financials.loc['Operating Income']

        if 'Depreciation' in financials.index:
            depreciation_amortization = financials.loc['Depreciation']
        else:
            depreciation_amortization = pd.Series(0, index=revenue.index)

        ebitda = operating_income + depreciation_amortization
        ebitda_margin = (ebitda / revenue) * 100
        return ebitda_margin.iloc[0]
    except:
        return None

# Calculate ROI and ROE
def fetch_roi_roe(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        financials = ticker.quarterly_financials
        balance_sheet = ticker.quarterly_balance_sheet

        net_income = financials.loc['Net Income']
        total_assets = balance_sheet.loc['Total Assets']
        total_equity = balance_sheet.loc["Total Stockholder Equity"]

        roi = (net_income / total_assets * 100).iloc[0]
        roe = (net_income / total_equity * 100).iloc[0]

        return roi, roe
    except:
        return None, None

# Combined Gradio function
def analyze_company(company_name):
    ticker = company_dict[company_name]
    headlines = fetch_news(ticker)
    sentiment_counts, overall_sentiment = analyze_sentiment(headlines)
    profit_loss = fetch_profit_loss(ticker)
    ebitda_margin = fetch_ebitda_margin(ticker)
    roi, roe = fetch_roi_roe(ticker)

    sentiment_output = (
        f"📰 **Sentiment Analysis for {company_name} ({ticker})**\n"
        f"Positive: {sentiment_counts['positive']}\n"
        f"Neutral: {sentiment_counts['neutral']}\n"
        f"Negative: {sentiment_counts['negative']}\n"
        f"Overall: {overall_sentiment}\n"
    )

    pl_output = (
        f"\n💹 **Profit/Loss for last 3 months**: "
        f"{profit_loss:.2f}%" if profit_loss is not None else "\n💹 No price data available."
    )

    ebitda_output = (
        f"\n📈 **Latest EBITDA Margin**: {ebitda_margin:.2f}%" if ebitda_margin is not None else "\n📈 EBITDA margin not available."
    )

    roi_output = (
        f"\n💰 **Return on Investment (ROI)**: {roi:.2f}%" if roi is not None else "\n💰 ROI not available."
    )

    roe_output = (
        f"\n🏦 **Return on Equity (ROE)**: {roe:.2f}%" if roe is not None else "\n🏦 ROE not available."
    )

    news_output = "\n\n🗞 **Top News Headlines:**\n" + "\n".join([f"- {h}" for h in headlines[:5]]) if headlines else "\n\n🗞 No recent headlines found."

    return sentiment_output + pl_output + ebitda_output + roi_output + roe_output + news_output

# Gradio app
with gr.Blocks() as demo:
    gr.Markdown("## 📊 Company Market Sentiment & Financial Metrics Dashboard")
    company_dropdown = gr.Dropdown(
        choices=list(company_dict.keys()),
        label="Select a Company",
        interactive=True
    )
    output_box = gr.Textbox(label="Analysis Output", lines=25, interactive=False)
    company_dropdown.change(fn=analyze_company, inputs=company_dropdown, outputs=output_box)

demo.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8d68b755904c0d7eb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [71]:
import gradio as gr
from transformers import pipeline
import yfinance as yf
import pandas as pd

# ... (rest of your existing code) ...

# Gradio app
with gr.Blocks() as demo:
    gr.Markdown("## 📊 Company Market Sentiment & Financial Metrics Dashboard")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📰 **Sentiment Analysis**")
            company_dropdown = gr.Dropdown(
                choices=list(company_dict.keys()),
                label="Select a Company for Sentiment",
                interactive=True
            )
            sentiment_output = gr.Textbox(label="Sentiment Analysis Output", lines=15, interactive=False)
            company_dropdown.change(fn=lambda x: analyze_company(x)[0], inputs=company_dropdown, outputs=sentiment_output)

        with gr.Column():
            gr.Markdown("### 📊 **Financial Metrics**")
            financial_output = gr.Textbox(label="Financial Metrics Output", lines=25, interactive=False)
            company_dropdown.change(fn=lambda x: analyze_company(x)[1], inputs=company_dropdown, outputs=financial_output)

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 🗞 **Latest News Headlines**")
            news_output = gr.Textbox(label="News Output", lines=10, interactive=False)
            company_dropdown.change(fn=lambda x: analyze_company(x)[2], inputs=company_dropdown, outputs=news_output)

        with gr.Column():
            gr.Markdown("### 🧠 **Investment Advice**")
            advice_output = gr.Textbox(label="Buy/Sell Recommendation", lines=3, interactive=False)
            company_dropdown.change(fn=lambda x: analyze_company(x)[3], inputs=company_dropdown, outputs=advice_output)
        # Removed the st.markdown lines as they were causing the error and st was not imported
        # If you intended to use st (streamlit), make sure it is installed and imported
        # and that the indentation is correct.

    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://996d384c6de6a77124.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
